# CS211: Data Privacy
## Homework 8

In [101]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
import math
from scipy import stats
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('https://github.com/jnear/cs211-data-privacy/raw/master/homework/adult_with_pii.csv')

## Range Queries

A *range query* counts the number of rows in the dataset which have a value lying in a given range. For example, "how many participants are between the ages of 21 and 33?" is a range query. A *workload* of range queries is just a list of range queries. The code below generates 100 random range queries over ages in the adult dataset.

In [37]:
def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])

random_lower_bounds = [random.randint(1, 70) for _ in range(100)]
random_workload = [(lb, random.randint(lb, 100)) for lb in random_lower_bounds]
real_answers = [range_query(adult, 'Age', lb, ub) for (lb, ub) in random_workload]
print('First 5 queries: ', random_workload[:5])

First 5 queries:  [(61, 85), (3, 56), (70, 91), (18, 99), (37, 67)]


## Question 1 (10 points)

Write code to answer a workload of range queries using `laplace_mech` and sequential composition. Your solution should have a **total privacy cost of epsilon**.

In [97]:
def workload_laplace(workload, epsilon): #no parallel comp, counting queries of sens 1
    ans_list = []#apply range query
    for work in workload: #seq comp
        a, b = work
        rng = range_query(adult, 'Age', a, b)
        noised = laplace_mech(rng, len(workload), epsilon)
        ans_list.append(noised)
    return ans_list
    #raise NotImplementedError()

print('First 4 answers:', workload_laplace(random_workload, 1.0)[:4])

First 4 answers: [2070.298298573069, 28363.796363845526, 586.5751484712308, 32263.844513765336]


In [98]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_laplace(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 50
assert np.mean(errors) < 200

Average absolute error: 86.28347300786741


## Question 2 (10 points)

Write code to answer a workload using `laplace_mech_vec` - the version of the Laplace mechanism for **vector-valued** queries. Your solution should *not* use sequential composition, and should have a total privacy cost of `epsilon`.

*Hint*: remember to use L1 global sensitivity.

In [93]:
def workload_laplace_vec(workload, epsilon): #no seq com, no parallel comp, adv if 70 <?
    ans_list = []
    L1 = len(workload)
    #L1 sens is sum of vector sens
    rng = [range_query(adult, 'Age', work[0], work[1]) for work in workload] #1st of tuple
    
    noise = laplace_mech_vec(rng, L1, epsilon)
    return noise
    #raise NotImplementedError()

print('First 4 answers:', workload_laplace_vec(random_workload, 1.0)[:4])

First 4 answers: [2159.0858378556854, 28532.704752057376, 690.6213155370223, 32166.855225662974]


In [94]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_laplace_vec(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 50
assert np.mean(errors) < 200

Average absolute error: 90.37845525111432


## Question 3 (10 points)

In 2-5 sentences, answer the following:
- Did the two solutions differ in terms of their accuracy?
- How do they differ in terms of their use of composition properties of differential privacy?

The two solutions' accuracy differ slightly with Q2 having less error. In compositiion, Q1 uses sequential composition on each query while Q2 applies this onto the vector of ranges.

## Question 3 (10 points)

Write code to answer a workload using `gaussian_mech_vec` - the version of the Gaussian mechanism for vector-valued queries. Your solution should not use sequential composition, should satisfy $(\epsilon, \delta)$-differential privacy, and should have a total privacy cost of (`epsilon`, `delta`).

*Hint*: remember to use L2 sensitivity.

In [105]:
def workload_gaussian_vec(workload, epsilon, delta):
    ans_list = []
    L2 = math.sqrt(len(workload))
    #L1 sens is sum of vector sens
    rng = [range_query(adult, 'Age', work[0], work[1]) for work in workload] #1st of tuple
    
    noise = gaussian_mech_vec(rng, L2, epsilon, delta)
    return noise
    raise NotImplementedError()

print('First 4 answers:', workload_gaussian_vec(random_workload, 1.0, 1e-5)[:4])

First 4 answers: [2251.3965735086917, 28452.37560998159, 547.5340181884806, 32161.050794808296]


In [153]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_gaussian_vec(random_workload, 1.0, 1e-5))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 10
assert np.mean(errors) < 100

Average absolute error: 35.17008939190504


## Question 4 (10 points)

In 2-5 sentences, answer the following:
- Of your solutions in questions 1-3, which ones rely on *sequential composition*?
- Which solution offers the best accuracy?
- Why does this particular solution yield the best accuracy?

Of the three solutions, the 1st relies most on sequential composition. In terms of accuracy the 3rd solution is best, being twice as better than the 2nd and three times better than the 1st. I believe this is due to 

## Question 5 (10 points)

Re-implement your solution to question 3 using *Rényi differential privacy*. Your solution should satisfy $(\alpha, \bar\epsilon)$-RDP.

*Hint*: see the "variants" chapter in the textbook.

In [122]:
def workload_gaussian_vec_RDP(workload, alpha, epsilon_bar):
    L2 = math.sqrt(len(workload))
    #L1 sens is sum of vector sens
    rng = [range_query(adult, 'Age', work[0], work[1]) for work in workload] #1st of tuple
    sigma = np.sqrt((L2**2 * alpha) / (2 * epsilon_bar))
    
    ans_list = [rn + np.random.normal(loc=0, scale=sigma) for rn in rng]
    
    return ans_list
    raise NotImplementedError()

print('First 4 answers:', workload_gaussian_vec(random_workload, 1.0, 1e-5)[:4])

First 4 answers: [2314.606216705241, 28476.498675927614, 663.9439319758189, 32154.36186790027]


In [123]:
# TEST CASE
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_gaussian_vec_RDP(random_workload, 5, 0.1))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 10
assert np.mean(errors) < 100

Average absolute error: 43.05192739414261


## Question 6 (10 points)

Implement a function `convert_RDP_ED` to convert from the $(\alpha, \bar\epsilon)$ of Rényi differential privacy to the $(\epsilon, \delta)$ of approximate differential privacy. Your function should also take the desired value of $\delta$.

In [128]:
def convert_RDP_ED(alpha, epsilon_bar, delta):
    #morph alpha, ep_bar t
    epsilon = epsilon_bar + (np.log(1/delta)/(alpha - 1))
    return epsilon
     
    raise NotImplementedError()

convert_RDP_ED(5, 0.1, 1e-5)

2.9782313662425572

In [146]:
convert_RDP_ED(12, .01, 1e-5)

1.0566295877245662

In [154]:
# TEST CASE
assert convert_RDP_ED(5, 0.1, 1e-5) == 2.9782313662425572
assert convert_RDP_ED(40, 0.1, 1e-5) == 0.39520321705051864
assert convert_RDP_ED(500, 1.0, 1e-5) == 1.02307199491978
assert convert_RDP_ED(40, 1.0, 1e-5) == 1.2952032170505188

In [152]:
errors = [abs(r_a - l_a) for (r_a, l_a) in 
          zip(real_answers, 
              workload_gaussian_vec_RDP(random_workload, 16, 1.0))]
print('Average absolute error:', np.mean(errors))

Average absolute error: 21.009214258884075


## Question 7 (10 points)

In 2-5 sentences, answer the following:
- Try various values for `alpha` and `epsilon_bar` in `convert_RDP_ED`. At what values do you observe an $(\epsilon, \delta)$ value around $(1.0, 10^{-5})$?
- Try these values for `alpha` and `epsilon_bar` in `workload_gaussian_vec_RDP`. How does the error compare to using `workload_gaussian_vec`?
- Is it useful to use Rényi differential privacy to answer workloads of range queries? Or is regular $(\epsilon, \delta)$-differential privacy just as good?

With a high alpha and low epsilon_bar the values begin to move closer to an epsilon of (1.0, 10^-5). After trying similar combinations on the workload gaussian vec RDP, it seems to be a bit more erroneous than against workload gaussian vec. Comparatively, it seems better to use regular (𝜖,𝛿) differential privacy. 